In [1]:
from IPython.display import display, SVG
import numpy as np
import pydot

import pydrake.all
from pydrake.all import *
from pydrake.all import (AddMultibodyPlantSceneGraph, AngleAxis,
                         DiagramBuilder, FindResourceOrThrow, Integrator,
                         JacobianWrtVariable, LeafSystem, MeshcatVisualizer,
                         MultibodyPlant, MultibodyPositionToGeometryPose,
                         Parser, PiecewisePolynomial, PiecewisePose,
                         Quaternion, Rgba, RigidTransform, RotationMatrix,
                         SceneGraph, Simulator, StartMeshcat, TrajectorySource, RevoluteJoint)


from pydrake.visualization import ModelVisualizer
from manipulation import running_as_notebook
from manipulation.meshcat_utils import MeshcatPoseSliders
from manipulation.scenarios import AddIiwa, AddShape, AddWsg, MakeManipulationStation


In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://ad97c8f3-e195-42b3-b69a-0eace2a495f9.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


In [3]:
class PseudoInverseController(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_WG", 6)
        self.q_port = self.DeclareVectorInputPort("iiwa_position", 7)
        self.DeclareVectorOutputPort("iiwa_velocity", 7, self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context, output):
        V_G = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kV,
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,self.iiwa_start:self.iiwa_end+1] # Only iiwa terms.
        v = np.linalg.pinv(J_G).dot(V_G)
        output.SetFromVector(v)

In [4]:
def MakeGripperFrames(X_G, X_O):
    """
    Takes a partial specification with X_G["initial"] and X_O["initial"] and
    X_0["goal"], and returns a X_G and times with all of the pick and place
    frames populated.
    """
    # Define (again) the gripper pose relative to the object when in grasp.
    p_GgraspO = [0, 0.11, 0]
    R_GgraspO = RotationMatrix.MakeXRotation(
        np.pi / 2.0) @ RotationMatrix.MakeZRotation(np.pi / 2.0)
    X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
    X_OGgrasp = X_GgraspO.inverse()
    # pregrasp is negative y in the gripper frame (see the figure!).
    X_GgraspGpregrasp = RigidTransform([0, -0.08, 0])

    X_G["pick"] = X_O["initial"] @ X_OGgrasp
    X_G["prepick"] = X_G["pick"] @ X_GgraspGpregrasp
    X_G["place"] = X_O["goal"] @ X_OGgrasp
    X_G["preplace"] = X_G["place"] @ X_GgraspGpregrasp

    # I'll interpolate a halfway orientation by converting to axis angle and halving the angle.
    X_GprepickGpreplace = X_G["prepick"].inverse() @ X_G["preplace"]
    angle_axis = X_GprepickGpreplace.rotation().ToAngleAxis()
    X_GprepickGclearance = RigidTransform(
        AngleAxis(angle=angle_axis.angle() / 2.0, axis=angle_axis.axis()),
        X_GprepickGpreplace.translation() / 2.0 + np.array([0, -0.3, 0]))
    X_G["clearance"] = X_G["prepick"] @ X_GprepickGclearance

    # Now let's set the timing
    times = {"initial": 0}
    X_GinitialGprepick = X_G["initial"].inverse() @ X_G["prepick"]
    times["prepick"] = times["initial"] + 10.0 * np.linalg.norm(
        X_GinitialGprepick.translation())
    # Allow some time for the gripper to close.
    times["pick_start"] = times["prepick"] + 2.0
    times["pick_end"] = times["pick_start"] + 2.0
    X_G["pick_start"] = X_G["pick"]
    X_G["pick_end"] = X_G["pick"]
    times["postpick"] = times["pick_end"] + 2.0
    X_G["postpick"] = X_G["prepick"]
    time_to_from_clearance = 10.0 * np.linalg.norm(
        X_GprepickGclearance.translation())
    times["clearance"] = times["postpick"] + time_to_from_clearance
    times["preplace"] = times["clearance"] + time_to_from_clearance
    times["place_start"] = times["preplace"] + 2.0
    times["place_end"] = times["place_start"] + 2.0
    X_G["place_start"] = X_G["place"]
    X_G["place_end"] = X_G["place"]
    times["postplace"] = times["place_end"] + 2.0
    X_G["postplace"] = X_G["preplace"]

    return X_G, times

X_G = {
"initial":
    RigidTransform(RotationMatrix.MakeXRotation(-np.pi / 2.0),
                    [0, -0.25, 0.25])
}
X_O = {
    "initial":
        RigidTransform(RotationMatrix.MakeZRotation(np.pi / 2.0),
                       [-.2, -.75, 0.025]),
    "goal":
        RigidTransform(RotationMatrix.MakeZRotation(np.pi), [.75, 0, 0.025])
}
X_G, times = MakeGripperFrames(X_G, X_O)
print(f"Sanity check: The entire maneuver will take {times['postplace']} seconds to execute.")

Sanity check: The entire maneuver will take 30.905782746839105 seconds to execute.


In [5]:
def MakeGripperPoseTrajectory(X_G, times):
    """
    Constructs a gripper position trajectory from the plan "sketch".
    """

    sample_times = []
    poses = []
    for name in ["initial", "prepick", "pick_start", "pick_end", "postpick",
                 "clearance", "preplace", "place_start", "place_end",
                 "postplace"]:
        sample_times.append(times[name])
        poses.append(X_G[name])

    return PiecewisePose.MakeLinear(sample_times, poses)

traj_X_G = MakeGripperPoseTrajectory(X_G, times)

In [6]:
def MakeGripperCommandTrajectory(times):
    opened = np.array([0.107]);
    closed = np.array([0.0]);

    traj_wsg_command = PiecewisePolynomial.FirstOrderHold(
        [times["initial"], times["pick_start"]], np.hstack([[opened], [opened]]))
    traj_wsg_command.AppendFirstOrderSegment(times["pick_end"], closed)
    traj_wsg_command.AppendFirstOrderSegment(times["place_start"], closed)
    traj_wsg_command.AppendFirstOrderSegment(times["place_end"], opened)
    traj_wsg_command.AppendFirstOrderSegment(times["postplace"], opened)
    return traj_wsg_command

traj_wsg_command = MakeGripperCommandTrajectory(times)

In [7]:
builder = DiagramBuilder()
# plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0001)
# parser = Parser(plant)

model_directives = """
directives:
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
- add_model:
    name: table_top
    file: file:///work/models/table_top.sdf
- add_weld:
    parent: world
    child: table_top_link
    X_PC:
        translation: [0, 0, 0]
- add_model:
    name: cupboard
    file: file:///work/models/cupboard.sdf
- add_weld:
    parent: world
    child: cupboard_body
    X_PC:
        translation: [1,0,0.409]
- add_weld:
    parent: world
    child: top_and_bottom
    X_PC:
        translation: [1,0,0.409]
- add_model:
    name: cracker_box
    file: package://drake/manipulation/models/ycb/sdf/003_cracker_box.sdf
"""

"""
# Load a cracker box from Drake. 
cracker_box = FindResourceOrThrow("drake/manipulation/models/ycb/sdf/003_cracker_box.sdf")
parser.AddModels(cracker_box)

# Load the table top and the cylinder we created.
parser.AddModelsFromString(cupboard_sdf, "sdf")
parser.AddModelsFromString(table_top_sdf, "sdf")

# Weld the table to the world so that it's fixed during the simulation.
cupboard_frame = plant.GetFrameByName("cupboard_body")
cupboard_frame2 = plant.GetFrameByName("top_and_bottom")
plant.WeldFrames(plant.world_frame(), cupboard_frame, RigidTransform(p=[1,0,0.409]))
plant.WeldFrames(plant.world_frame(), cupboard_frame2, RigidTransform(p=[1,0,0.409]))


# Weld the table to the world so that it's fixed during the simulation.
table_frame = plant.GetFrameByName("table_top_center")
plant.WeldFrames(plant.world_frame(), table_frame)
"""
station = builder.AddSystem(
    MakeManipulationStation(model_directives=model_directives))
plant = station.GetSubsystemByName("plant")
# Find the initial pose of the gripper (as set in the default Context)
temp_context = station.CreateDefaultContext()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)
"""
# add iiwa robot
iiwa = AddIiwa(plant, "with_box_collision")
wsg = AddWsg(plant, iiwa, welded=True)
"""

# Finalize the plant after loading the scene.
# plant.Finalize()
meshcat.Delete()
meshcat.DeleteAddedControls()
"""
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, 
    scene_graph, 
    meshcat,
    MeshcatVisualizerParams(role=Role.kPerception, prefix="visual"))
"""
# diagram = builder.Build()
# context = diagram.CreateDefaultContext()

X_O = {"initial": RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [-.2, -.65, 0.0]),
       "goal": RigidTransform(RotationMatrix.MakeZRotation(np.pi),[.5, 0, 0.0])}

# plant.SetDefaultFreeBodyPose(plant.GetBodyByName("base_link_cracker"), X_O['initial'])
# plant_context = plant.GetMyContextFromRoot(context)
X_G = {
    "initial":
        plant.EvalBodyPoseInWorld(temp_plant_context,
                                  plant.GetBodyByName("body"))
}
X_G, times = MakeGripperFrames(X_G, X_O)
print(f"Sanity check: The entire maneuver will take {times['postplace']} seconds to execute.")

# Make the trajectories
traj = MakeGripperPoseTrajectory(X_G, times)
traj_V_G = traj.MakeDerivative()

V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
V_G_source.set_name("v_WG")
controller = builder.AddSystem(PseudoInverseController(plant))
controller.set_name("PseudoInverseController")
builder.Connect(V_G_source.get_output_port(), controller.GetInputPort("V_WG"))

integrator = builder.AddSystem(Integrator(7))
integrator.set_name("integrator")
builder.Connect(controller.get_output_port(),
                integrator.get_input_port())
builder.Connect(integrator.get_output_port(),
                station.GetInputPort("iiwa_position"))
builder.Connect(station.GetOutputPort("iiwa_position_measured"),
                controller.GetInputPort("iiwa_position"))

traj_wsg_command = MakeGripperCommandTrajectory(times)
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg_command))
wsg_source.set_name("wsg_command")
builder.Connect(wsg_source.get_output_port(), station.GetInputPort("wsg_position"))

meshcat.Delete()
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, station.GetOutputPort("query_object"), meshcat)

diagram = builder.Build()
diagram.set_name("pick_and_place")

simulator = Simulator(diagram)
context = simulator.get_mutable_context()
station_context = station.GetMyContextFromRoot(context)
integrator.set_integral_value(
    integrator.GetMyContextFromRoot(context),
    plant.GetPositions(plant.GetMyContextFromRoot(context),
                       plant.GetModelInstanceByName("iiwa")))

traj_p_G = traj_X_G.get_position_trajectory()

p_G = traj_p_G.vector_values(traj_p_G.get_segment_times())

traj_v_G = traj_p_G.MakeDerivative()

v_G = traj_v_G.vector_values(traj_v_G.get_segment_times())

visualizer.StartRecording(False)
simulator.AdvanceTo(traj_p_G.end_time() if running_as_notebook else 0.1)
visualizer.PublishRecording()

# We use the default context to calculate the transformation of the table
# in world frame but this is NOT the context the Diagram consumes.
plant_context = plant.CreateDefaultContext()

"""
cracker_box = plant.GetBodyByName("base_link_cracker")
X_TableCracker = RigidTransform(
    RollPitchYaw(np.asarray([0, 0, 0]) * np.pi / 180), p=[1,0,0.6])
plant.SetDefaultFreeBodyPose(cracker_box, X_TableCracker)
"""


simulator = Simulator(diagram)
simulator.Initialize()
simulator.set_target_realtime_rate(1.)
visualizer.StartRecording()
simulator.AdvanceTo(5.0)
visualizer.PublishRecording()

Sanity check: The entire maneuver will take 28.880346490117844 seconds to execute.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ad97c8f3-e195-42b3-b69a-0eace2a495f9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>